<a href="https://colab.research.google.com/github/entanglement-nak/portfolio-nak/blob/main/CNN_memory_profiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install memory_profiler

In [ ]:
%load_ext memory_profiler

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from keras.utils import to_categorical
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
from memory_profiler import profile
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# データの読み込みと前処理
# profile
def load_and_preprocess_data(train_path, test_path):
    # データの読み込み
    train_df = pd.read_excel(train_path)
    test_df = pd.read_excel(test_path)

    # データの結合
    combined_data = pd.concat([train_df, test_df])
    y = combined_data.iloc[:, 0]
    X = combined_data.iloc[:, 1:]

    # 特徴量の列名を文字列に変換
    X.columns = X.columns.astype(str)

    # 特徴量のデータを数値型に変換し、欠損値を埋める
    X_cleaned = X.apply(pd.to_numeric, errors='coerce').fillna(0)

    # 特徴量の正規化
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(X_cleaned)

    return X_normalized, y

In [ ]:
# CNNモデルの構築
# profile
def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# メイン関数
def main():
    # データパス
    train_path = r"/content/drive/MyDrive/Wafer/Wafer_TRAIN.xlsx"
    test_path = r"/content/drive/MyDrive/Wafer/Wafer_TEST.xlsx"

    # データの読み込みと前処理
    X, y = load_and_preprocess_data(train_path, test_path)

    # データの分割
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # データの形状調整
    X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # ラベルのエンコーディング
    y_train_encoded = (y_train + 1) / 2
    y_test_encoded = (y_test + 1) / 2

    # CNNモデルの構築
    model = build_cnn_model((X_train.shape[1], 1))

    # モデル訓練の時間計測を開始
    start_train_time = time.time()
    model.fit(X_train_reshaped, y_train_encoded, epochs=10, validation_data=(X_test_reshaped, y_test_encoded))
    # モデル訓練の時間計測を終了
    end_train_time = time.time()
    train_elapsed_time = end_train_time - start_train_time

    # 予測時間の計測を開始
    start_predict_time = time.time()
    y_pred = model.predict(X_test_reshaped)
    # 予測時間の計測を終了
    end_predict_time = time.time()
    predict_elapsed_time = end_predict_time - start_predict_time

    y_pred = (y_pred > 0.5).astype(int).flatten()

    # 評価指標の計算
    accuracy = accuracy_score(y_test_encoded, y_pred)
    f1_macro = f1_score(y_test_encoded, y_pred, average='macro')
    f1_weighted = f1_score(y_test_encoded, y_pred, average='weighted')
    mcc = matthews_corrcoef(y_test_encoded, y_pred)

    # 結果の出力
    print(f'Accuracy: {accuracy}')
    print(f'Macro F1 Score: {f1_macro}')
    print(f'Weighted F1 Score: {f1_weighted}')
    print(f'MCC: {mcc}')
    print(f"予測にかかった時間: {predict_elapsed_time} 秒")

In [ ]:
# メモリの計測
for i in range(10):
    print(f"{i+1}回目の計測")
    %memit main()

1回目の計測
Epoch 1/10
157/157 [==============================] - 2s 12ms/step - loss: 0.1211 - accuracy: 0.9515 - val_loss: 0.0710 - val_accuracy: 0.9940
Epoch 2/10
157/157 [==============================] - 2s 14ms/step - loss: 0.0682 - accuracy: 0.9924 - val_loss: 0.0620 - val_accuracy: 0.9963
Epoch 3/10
157/157 [==============================] - 2s 11ms/step - loss: 0.0609 - accuracy: 0.9954 - val_loss: 0.0562 - val_accuracy: 0.9977
Epoch 4/10
157/157 [==============================] - 2s 11ms/step - loss: 0.0493 - accuracy: 0.9980 - val_loss: 0.0091 - val_accuracy: 0.9981
Epoch 5/10
157/157 [==============================] - 2s 10ms/step - loss: 0.0122 - accuracy: 0.9974 - val_loss: 0.0152 - val_accuracy: 0.9967
Epoch 6/10
157/157 [==============================] - 2s 10ms/step - loss: 0.0117 - accuracy: 0.9974 - val_loss: 0.0072 - val_accuracy: 0.9986
Epoch 7/10
157/157 [==============================] - 1s 9ms/step - loss: 0.0047 - accuracy: 0.9990 - val_loss: 0.0063 - val_accuracy: 